In [1]:
import requests
from pprint import pprint
from bs4 import BeautifulSoup
from time import sleep
import json
import os
import pandas as pd

## Web Scraping for COVID Data: USAFacts.org

In [2]:
#USAFacts site
url = 'https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/'
state_covid_page = requests.get(url)

In [3]:
state_covid_page.status_code

200

In [4]:
state_covid_soup = BeautifulSoup(state_covid_page.content, 'lxml')

In [5]:
print(state_covid_soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <title data-react-helmet="true">
   Coronavirus Locations: COVID-19 Map by County and State | USAFacts
  </title>
  <meta content="IE=edge" data-react-helmet="true" http-equiv="X-UA-Compatible"/>
  <meta charset="utf-8" data-react-helmet="true"/>
  <meta content="width=device-width, initial-scale=1" data-react-helmet="true" name="viewport"/>
  <meta content="en" data-react-helmet="true" property="og:locale"/>
  <meta content="#015193" data-react-helmet="true" name="theme-color"/>
  <meta content="Follow new cases found each day and the total number of cases and deaths in the US. The county-level tracker makes it easy to follow COVID-19 cases on a granular level, as does the ability to break down infections per 100,000 people." data-react-helmet="true" name="description"/>
  <meta content="" data-react-helmet="true" name="keywords"/>
  <meta content="summary_large_image" data-react-helmet="true" name="twitter:card"/>
  <meta content="@usafacts"

In [6]:
title = state_covid_soup.find('title')
print(title.text)

Coronavirus Locations: COVID-19 Map by County and State | USAFacts


In [7]:
# We want to get data from the table for each state.
all_tables = state_covid_soup.find_all('table')
# all_tables
len(all_tables)

1

In [8]:
# Extract the table.
the_table = all_tables[0]
# type(the_table)
the_table

<table aria-label="a table of data" class="MuiTable-root-45 jss42"><thead class="MuiTableHead-root-47"><tr class="MuiTableRow-root-48 MuiTableRow-head-50"><th class="MuiTableCell-root-52 MuiTableCell-head-53 jss43 MuiTableCell-sizeSmall-56" scope="col">State</th><th class="MuiTableCell-root-52 MuiTableCell-head-53 jss43 MuiTableCell-sizeSmall-56" scope="col">Confirmed</th><th class="MuiTableCell-root-52 MuiTableCell-head-53 jss43 MuiTableCell-sizeSmall-56" scope="col">Deaths</th></tr></thead><tbody class="MuiTableBody-root-64"><tr class="MuiTableRow-root-48"><th class="MuiTableCell-root-52 MuiTableCell-body-54 jss43 MuiTableCell-sizeSmall-56" role="cell" scope="row"><a aria-current="page" class="jss41" href="/visualizations/coronavirus-covid-19-spread-map/state/alabama">Alabama</a></th><td class="MuiTableCell-root-52 MuiTableCell-body-54 MuiTableCell-sizeSmall-56" role="cell" scope="col">82,366</td><td class="MuiTableCell-root-52 MuiTableCell-body-54 MuiTableCell-sizeSmall-56" role="ce

In [9]:
p = state_covid_soup.find('p', class_='jss26')
string_list= p.text.split()
month = string_list[2]
day = string_list[3].strip()
year = string_list[-1]
the_date = f'{year}-{month}-{day}'.split(',')
date = the_date[0]
date

'2020-Jul-30'

In [10]:
# Lets get the name of each state.
#states = [a.string for a in the_table.find_all('a')]
#len(states)

In [11]:
# Extract the row of data
date_list = []
state_list = []
confirmed_state_list = []
deaths_state_list = []

all_trs = [tr for tr in the_table.find_all('tr')]

for tr in all_trs[1:]:
    # Name of state
    state = tr.a.text
    state_list.append(state)
    
    # COVID cases and deaths
    all_tds = [td for td in tr.find_all('td')]
    confirmed = all_tds[0].text
    deaths = all_tds[1].text
    confirmed_state_list.append(confirmed)
    deaths_state_list.append(deaths)
    
    date_list.append(date)

In [12]:
# Create a dictionary with the information.
covid_state_dict = {}
covid_state_dict["state"] = state_list
covid_state_dict["confirmed"] = confirmed_state_list
covid_state_dict["deaths"] = deaths_state_list
covid_state_dict["date"] = date_list

## COVID: State Level

In [13]:
# Create the DataFrame.
covid_state_df = pd.DataFrame(data = covid_state_dict)
covid_state_df #.head()

,state,confirmed,deaths,date
0,Alabama,"82,366","1,538",2020-Jul-30
1,Alaska,"2,798",22,2020-Jul-30
2,Arizona,"168,273","3,454",2020-Jul-30
3,Arkansas,"40,968",434,2020-Jul-30
4,California,"483,512","8,908",2020-Jul-30
5,Colorado,"45,796","1,822",2020-Jul-30
6,Connecticut,"49,540","4,425",2020-Jul-30
7,Delaware,"14,689",581,2020-Jul-30
8,District of Columbia,"12,057",584,2020-Jul-30
9,Florida,"461,379","6,586",2020-Jul-30


## Now scrap the url for each county in the US

In [14]:
# These are the url's for each state to link to the HTML for the county level data.
base_url = 'https://usafacts.org'
all_state_url = [base_url+a['href'] for a in the_table.find_all('a')]
#all_state_url

In [15]:
# Initialize the lists for the Pandas DateFrame.
date_county_list = []
county_list = []
the_state_list =[]
confirmed_county_list = []
deaths_county_list = []
confirmed_per_100k_list = []

# Loop through every county of each state.
for state_url in all_state_url:
    state_county_page = requests.get(state_url)
    state_county_soup = BeautifulSoup(state_county_page.content, 'lxml')
    
    # The date that data was extracted.
    string_county_list= state_county_soup.find('p').text.split()
    month_county = string_county_list[2]
    day_county = string_county_list[3].strip()
    year_county = string_county_list[-1]
    the_date_county = f'{year}-{month}-{day}'.split(',')
    date_county = the_date[0]
    
    # Parse the state_county_soup for table information.
    all_county_tables = state_county_soup.find_all('table')
    #len(all_county_tables)

    # Extract the table.
    the_county_table = all_county_tables[0]
    # type(the_table)

    all_county_trs = [tr for tr in the_county_table.tbody.find_all('tr')]

    # Extract the state name from the url.
    state_of_county = all_county_trs[0].a['href'].split('/')[-3]
    
    for county_tr in all_county_trs:
        # Name of each county.
        county = county_tr.a.text
        county_list.append(county)
        
        # County COVID cases, deaths and Confirmed Per 100k
        all_county_tds = [td for td in county_tr.find_all('td')]
        county_confirmed = all_county_tds[0].text
        county_deaths = all_county_tds[1].text
        county_confirmed_per_100k = all_county_tds[2].text
        confirmed_county_list.append(county_confirmed)
        deaths_county_list.append(county_deaths)
        confirmed_per_100k_list.append(county_confirmed_per_100k)

        the_state_list.append(state_of_county.capitalize())
        date_county_list.append(date_county)
    
    # Create a dictionary with the information.
    covid_county_dict = {}
    covid_county_dict["county"] = county_list
    covid_county_dict["state"] = the_state_list
    covid_county_dict["confirmed"] = confirmed_county_list
    covid_county_dict["deaths"] = deaths_county_list
    covid_county_dict["confirmed_per_100k"] = confirmed_per_100k_list
    covid_county_dict["date"] = date_county_list
    
    #break

## COVID: County Level

In [16]:
# Create the DataFrame.
covid_county_df = pd.DataFrame(data = covid_county_dict)
covid_county_df.head()

,county,state,confirmed,deaths,confirmed_per_100k,date
0,Autauga County,Alabama,974,21,"1,743.4",2020-Jul-30
1,Baldwin County,Alabama,"2,835",21,"1,270.0",2020-Jul-30
2,Barbour County,Alabama,575,4,"2,329.3",2020-Jul-30
3,Bibb County,Alabama,338,2,"1,509.3",2020-Jul-30
4,Blount County,Alabama,675,3,"1,167.3",2020-Jul-30


In [17]:
covid_county_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3142 entries, 0 to 3141
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   county              3142 non-null   object
 1   state               3142 non-null   object
 2   confirmed           3142 non-null   object
 3   deaths              3142 non-null   object
 4   confirmed_per_100k  3142 non-null   object
 5   date                3142 non-null   object
dtypes: object(6)
memory usage: 147.4+ KB


## COVID: Race/Ethnicity Demographics
### The COVID Tracking Project:  The COVID Racial Data Tracker.

In [18]:
# Load the demographic data Race_Data _Entry _CRDT_utf_8 .csv file.
the_filepath = os.path.join(".", "Resources", "Race_Data _Entry _CRDT_utf_8 .csv")

# Read file and store into Pandas DataFrames
covid_race_df = pd.read_csv(the_filepath, encoding = 'utf-8')

# Visualize
covid_race_df.head()

,date,state,cases_total,cases_white,cases_black,cases_latinX,cases_asian,cases_AIAN,cases_NHPI,cases_multiracial,...,deaths_latinX,deaths_asian,deaths_AIAN,deaths_NHPI,deaths_multiracial,deaths_other,deaths_unknown,deaths_ethnicity_hispanic,deaths_ethnicity_non_hispanic,deaths_ethnicity_unknown
0,20200729,AK,2797,852,81.0,NaN,83.0,366.0,103.0,68.0,...,NaN,2.0,8.0,1.0,0.0,0.0,0.0,0.0,22.0,0.0
1,20200729,AL,83782,25915,23239.0,NaN,309.0,NaN,NaN,NaN,...,NaN,4.0,NaN,NaN,NaN,29.0,103.0,48.0,1267.0,223.0
2,20200729,AR,40968,20527,8673.0,NaN,587.0,120.0,2103.0,NaN,...,NaN,6.0,2.0,31.0,NaN,30.0,0.0,43.0,394.0,0.0
3,20200729,AS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20200729,AZ,168273,29227,3822.0,38887.0,1309.0,9631.0,NaN,NaN,...,946.0,42.0,430.0,NaN,NaN,57.0,503.0,946.0,2005.0,503.0


## Using World Population Review API: JSON

In [19]:
# Set up the URL will be using to retrieve the infromation as JSON.
query_url = "https://worldpopulationreview.com/static/states/abbr-name-list.json"

# Get data from API: World Population Review
response = requests.get(query_url)
response_json = response.json()

In [20]:
len(response_json)

51

In [21]:
# Initialize the lists
state_name_list = []
abbr_list = []
abbr_dict = {}

for result in response_json:
    state = result['name']
    abbr = result['abbreviation']
    
    # Append the lists.
    state_name_list.append(state)
    abbr_list.append(abbr)
    
# Create a dictionary with the information.    
abbr_dict["state"] = state_name_list
abbr_dict["abbr"] = abbr_list

# Create the DataFrame.
abbreviation_df = pd.DataFrame(data = abbr_dict)
abbreviation_df.head()

,state,abbr
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


## United States Census Bureau

In [22]:
# Load the Census_Data_utf.csv file that was create Census Bureau.
filepath = os.path.join(".", "Resources", "Census_Data_utf.csv")

# Read file and store into Pandas DataFrames
census_data_df = pd.read_csv(filepath, encoding = 'utf-8')

# Visualize
census_data_df.head()

,FIPS,Geog_Level,State_Code,State_name,County_code,County_name,Lat_State,Lon_State,LAND_AREA,Med_HHD_Inc_ACS_14_18($),...,pct_Pov_Univ_ACS_14_18,pct_Prs_Blw_Pov_Lev_ACS_14_18,pct_One_Health_Ins_ACS_14_18,pct_No_Health_Ins_ACS_14_18,pct_NoHealthIns_65P_ACS_14_18,pct_Pop_NoCompDevic_ACS_14_18,pct_Pop_w_BroadComp_ACS_14_18,pct_HHD_NoCompDevic_ACS_14_18,pct_HHD_No_Internet_ACS_14_18,pct_HHD_w_Broadband_ACS_14_18
0,1000,State,1,Alabama,0,All counties,32.7794,-86.8287,50645.319,48486.0,...,97.48,17.49,69.93,9.82,0.41,11.71,77.81,16.61,22.40,56.41
1,1001,County,1,Alabama,1,Autauga County,NaN,NaN,594.437,58786.0,...,99.21,15.38,72.36,7.02,0.00,8.18,84.00,13.01,19.03,61.97
2,1003,County,1,Alabama,3,Baldwin County,NaN,NaN,1589.786,55962.0,...,98.47,10.57,69.54,10.03,0.41,7.21,83.03,11.43,16.83,57.24
3,1005,County,1,Alabama,5,Barbour County,NaN,NaN,884.875,34186.0,...,88.65,28.86,58.07,9.92,0.29,18.02,64.92,23.95,34.11,38.84
4,1007,County,1,Alabama,7,Bibb County,NaN,NaN,622.581,45340.0,...,90.86,13.99,65.90,7.19,0.00,17.20,72.83,23.73,29.24,33.42


## FIPS State

In [27]:
# Load the Census_Data_utf.csv file that was create Census Bureau.
state_filepath = os.path.join(".", "Resources", "state_centroid_utf_8.csv")

# Read file and store into Pandas DataFrames
fips_state_df = pd.read_csv(state_filepath, encoding = 'utf-8')

# Visualize
fips_state_df.head()

,state,fips_state,latitude,longitude
0,Alabama,1.0,32.7794,-86.8287
1,Alaska,2.0,64.0685,-152.2782
2,Arizona,4.0,34.2744,-111.6602
3,Arkansas,5.0,34.8938,-92.4426
4,California,6.0,37.1841,-119.4696


## FIPS County

In [24]:
# Load the Census_Data_utf.csv file that was create Census Bureau.
county_filepath = os.path.join(".", "Resources", "county_centroid_utf_8.csv")

# Read file and store into Pandas DataFrames
fips_county_df = pd.read_csv(county_filepath, encoding = 'utf-8')

# Visualize
fips_county_df.head()

,fips_county,state,county,latitude,longitude
0,1001,AL,Autauga,32.536382,-86.6445
1,1003,AL,Baldwin,30.659218,-87.7461
2,1005,AL,Barbour,31.870670,-85.4055
3,1007,AL,Bibb,33.015893,-87.1271
4,1009,AL,Blount,33.977448,-86.5672


In [ ]:
all_county_tables = state_county_soup.find_all('table')
len(all_county_tables)
# Extract the table.
the_county_table = all_county_tables[0]
# type(the_table)

all_county_trs = [tr for tr in the_county_table.tbody.find_all('tr')]

# Extract the state name from the url.
state_of_county = all_county_trs[0].a['href'].split('/')[-3]

for county_tr in all_county_trs:
    # Name of county
    county = county_tr.a.text
    county_list.append(county)
    
    # County COVID cases, deaths and Confirmed Per 100k
    all_county_tds = [td for td in county_tr.find_all('td')]
    county_confirmed = all_county_tds[0].text
    county_deaths = all_county_tds[1].text
    county_confirmed_per_100k = all_county_tds[2].text
    confirmed_county_list.append(county_confirmed)
    deaths_county_list.append(county_deaths)
    confirmed_per_100k_list.append(county_confirmed_per_100k)
    
    the_state_list.append(state_of_county.capitalize())
    date_county_list.append(date_county)
        
# Create a dictionary with the information.
covid_county_dict = {}
covid_county_dict["county"] = county_list
covid_county_dict["state"] = the_state_list
covid_county_dict["confirmed"] = confirmed_county_list
covid_county_dict["deaths"] = deaths_county_list
covid_county_dict["confirmed_per_100k"] = confirmed_per_100k_list
covid_county_dict["date"] = date_county_list

# Create the DataFrame.
covid_county_df = pd.DataFrame(data = covid_county_dict)


covid_county_df   #.head()


In [ ]:
all_county_trs[0].a['href'].split('/')[-3].title()

In [ ]:
print(the_county_table.find('tr'))